In [3]:
import pandas as pd
import numpy as np
import math
import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, GRU

from itertools import cycle



In [4]:
bist100 = pd.read_csv("dataset.csv")
bist100.head()

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0


In [5]:
bist100['timestamp'] = pd.to_datetime(bist100.timestamp)
bist100.head()

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0


In [6]:
bist100.sort_values(by='timestamp', inplace=True)
bist100.head()

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0


In [7]:
bist100.shape

(17414, 10)

In [8]:
print("Starting date: ",bist100.iloc[0][-1])
print("Ending date: ", bist100.iloc[-1][-1])
print("Duration: ", bist100.iloc[-1][-1]-bist100.iloc[0][-1])

Starting date:  3.0
Ending date:  3.0
Duration:  0.0


C:\Users\SARVESH AP\AppData\Local\Temp\ipykernel_20556\2553818647.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("Starting date: ",bist100.iloc[0][-1])
C:\Users\SARVESH AP\AppData\Local\Temp\ipykernel_20556\2553818647.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("Ending date: ", bist100.iloc[-1][-1])
C:\Users\SARVESH AP\AppData\Local\Temp\ipykernel_20556\2553818647.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[

In [9]:
closedf = bist100[['timestamp','cnt']]
print("Shape of close dataframe:", closedf.shape)

Shape of close dataframe: (17414, 2)


In [10]:
close_stock = closedf.copy()
del closedf['timestamp']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

(17414, 1)


In [11]:
training_size=int(len(closedf)*0.65)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

train_data:  (11319, 1)
test_data:  (6095, 1)


In [12]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [13]:
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

X_train:  (11303, 15)
y_train:  (11303,)
X_test:  (6079, 15)
y_test (6079,)


In [14]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (11303, 15, 1)
X_test:  (6079, 15, 1)


In [15]:
tf.keras.backend.clear_session()
model=Sequential()
model.add(LSTM(32,return_sequences=True,input_shape=(time_step,1)))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

D:\anaconda 3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 15, 32)              │           4,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 15, 32)              │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 32)                  │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,025 (82.13 KB)

 Trainable params: 21,025 (82.13 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=5,verbose=1)

Epoch 1/20
2261/2261 ━━━━━━━━━━━━━━━━━━━━ 44s 15ms/step - loss: 0.0122 - val_loss: 0.0030
Epoch 2/20
2261/2261 ━━━━━━━━━━━━━━━━━━━━ 35s 16ms/step - loss: 0.0022 - val_loss: 0.0023
Epoch 3/20
2261/2261 ━━━━━━━━━━━━━━━━━━━━ 32s 14ms/step - loss: 0.0018 - val_loss: 0.0024
Epoch 4/20
2261/2261 ━━━━━━━━━━━━━━━━━━━━ 32s 14ms/step - loss: 0.0013 - val_loss: 0.0013
Epoch 5/20
2261/2261 ━━━━━━━━━━━━━━━━━━━━ 32s 14ms/step - loss: 9.6914e-04 - val_loss: 8.2910e-04
Epoch 6/20
2261/2261 ━━━━━━━━━━━━━━━━━━━━ 32s 14ms/step - loss: 8.8110e-04 - val_loss: 9.8748e-04
Epoch 7/20
2261/2261 ━━━━━━━━━━━━━━━━━━━━ 32s 14ms/step - loss: 8.3455e-04 - val_loss: 9.6232e-04
Epoch 8/20
2261/2261 ━━━━━━━━━━━━━━━━━━━━ 32s 14ms/step - loss: 6.9220e-04 - val_loss: 9.6439e-04
Epoch 9/20
2261/2261 ━━━━━━━━━━━━━━━━━━━━ 33s 15ms/step - loss: 6.8546e-04 - val_loss: 7.3009e-04
Epoch 10/20
2261/2261 ━━━━━━━━━━━━━━━━━━━━ 30s 13ms/step - loss: 6.2064e-04 - val_loss: 7.5594e-04
Epoch 11/20
2261/2261 ━━━━━━━━━━━━━━━━━━━━ 45s 20ms

In [ ]:
model.save('lstm_model.h5')
print("Model saved as 'lstm_model.h5'")

In [ ]:
import joblib

# Save the scaler to a file
scaler_filename = "scaler.save"
joblib.dump(scaler, scaler_filename)

In [ ]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

In [ ]:
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

In [ ]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Test data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
look_back=time_step
# Create an empty array for train predictions
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict) + look_back, :] = train_predict

# Create an empty array for test predictions
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict) + (look_back * 2) + 1:len(closedf) - 1, :] = test_predict
# Create DataFrame for plotting
plotdf = pd.DataFrame({'date': close_stock['timestamp'],
                       'original_bikerent': close_stock['cnt'],
                       'train_predicted_bikerent': trainPredictPlot.reshape(1, -1)[0].tolist(),
                       'test_predicted_bikerent': testPredictPlot.reshape(1, -1)[0].tolist()})

# Plot the data using matplotlib
plt.figure(figsize=(10,6))
plt.plot(plotdf['date'], plotdf['original_bikerent'], label='Original bikerent price', color='blue')
plt.plot(plotdf['date'], plotdf['train_predicted_bikerent'], label='Train predicted bikerent price', color='green')
plt.plot(plotdf['date'], plotdf['test_predicted_bikerent'], label='Test predicted bikerent price', color='red')

# Customize the plot
plt.title('Comparison between original bikerent price vs predicted bikerent price')
plt.xlabel('Date')
plt.ylabel('bikerent Price')
plt.legend(title='bikerent Price prediction')
plt.grid(True)

# Rotate the x-axis labels for better visibility if necessary
plt.xticks(rotation=45)

# Display the plot
plt.tight_layout()
plt.show()


In [ ]:
new_data = np.array([[400]])  # Replace 'your_value' with the actual data point

# Scale the new data point using the loaded scaler
scaled_data = scaler.transform(new_data)

# Prepare the data for the LSTM model (time_step should be the same as in training)
time_step = 15
if len(scaled_data) < time_step:
    padded_data = np.zeros((time_step, 1))
    padded_data[-len(scaled_data):] = scaled_data
else:
    padded_data = scaled_data[-time_step:]

X_single = padded_data.reshape(1, time_step, 1)

# Predict the value for the single data point
predicted_value = model.predict(X_single)
predicted_value = scaler.inverse_transform(predicted_value)  # Inverse scale to get actual value
print("Predicted value:", predicted_value[0][0])